# <span style="font-width:bold; font-size: 3rem; color:#1EB182;">**Hopsworks Feature Store** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Model training & UI Exploration</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/nyc_taxi_fares/4_model_training_and_registration.ipynb)


## 🗒️ This notebook is divided into 3 main sections:
1. **Loading the training data**,
2. **Train the model**,
3. **Register model to Hopsworks model registry**.

![tutorial-flow](../../images/03_model.png)

In [ ]:
# install some libs for streamlit app
!pip install streamlit_folium streamlit-folium --quiet

In [ ]:
import warnings

# Mute warnings
warnings.filterwarnings("ignore")

## <span style="color:#ff5f27;"> 📡 Connecting to the Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

---

## <span style="color:#ff5f27;">🪝 Feature View and Training Dataset Retrieval</span>

To retrieve training dataset from **Feature Store** you retrieve **Feature View** using `FeatureStore.get_feature_view` method.

Then you can use **Feature View** in order to retrieve **training dataset** using `FeatureView.get_training_dataset` method.


In [ ]:
nyc_fares_fv = fs.get_feature_view(
    name = 'nyc_taxi_fares_fv',
    version = 1
)

In [ ]:
X_train, X_test, y_train, y_test = nyc_fares_fv.get_train_test_split(
    training_dataset_version=1
)

In [ ]:
X_test.head(5)

In [ ]:
cols_to_drop = ['ride_id']

In [ ]:
X_train = X_train.drop(cols_to_drop, axis=1)
X_test = X_test.drop(cols_to_drop, axis=1)

In [ ]:
# import numpy as np


# y_train = np.ravel(y_train)
# y_test = np.ravel(y_test)

In [ ]:
y_test

---

## <span style="color:#ff5f27;">🧬 Modeling</span>

### <span style="color:#ff5f27;">📝 Importing Libraries</span>

In [ ]:
import pandas as pd


from sklearn.metrics import mean_absolute_error, r2_score

In [ ]:
!pip install -U xgboost --quiet

In [ ]:
import xgboost as xgb


regressor = xgb.XGBRegressor()

regressor.fit(X_train.values, y_train.values)

In [ ]:
y_pred = regressor.predict(X_test)

xgb_r2_score = r2_score(y_test, y_pred)
xgb_mae = mean_absolute_error(y_test, y_pred)

print("XGBRegressor R²:", xgb_r2_score)
print("XGBRegressor MAE:", xgb_mae)

metrics = {
    'mae': xgb_mae,
    'r2_score': xgb_r2_score
}


### Remember, the data is random, so the results are not accurate at all.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


df_ = pd.DataFrame({
    "y_true": y_test.total_fare.tolist(),
    "y_pred": y_pred
})

residplot = sns.residplot(data=df_, x="y_true", y="y_pred", color='#613F75')
plt.title('Model Residuals')
plt.xlabel('Obsevation #')
plt.ylabel('Error')

plt.show()

In [ ]:
import os

if os.path.isdir("assets") == False:
    os.mkdir("assets")


fig = residplot.get_figure()
fig.savefig("assets/residplot.png") 
fig.show()

---

### <span style="color:#ff5f27;">⚙️ Model Schema</span>

The model needs to be set up with a [Model Schema](https://docs.hopsworks.ai/machine-learning-api/latest/generated/model_schema/), which describes the inputs and outputs for a model.

A Model Schema can be automatically generated from training examples, as shown below.

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()

## <span style='color:#ff5f27'>🗄 Model Registry</span>

One of the features in Hopsworks is the model registry. This is where you can store different versions of models and compare their performance. Models from the registry can then be served as API endpoints.

In [ ]:
import joblib
import os
import shutil


# The 'nyc_taxi_fares_model' directory will be saved to the model registry
model_dir="nyc_taxi_fares_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

joblib.dump(regressor, model_dir + '/nyc_taxi_fares_model.pkl')

shutil.copyfile("assets/residplot.png", model_dir + "/residplot.png")

With the schema in place, you can finally register our model.

In [ ]:
mr = project.get_model_registry()

nyc_model = mr.python.create_model(
    name="nyc_taxi_fares_model", 
    metrics=metrics,
    model_schema=model_schema,
    input_example=X_train.sample().values, 
    description="NYC Taxi fares Predictor")

nyc_model.save(model_dir)

It's important to know that every time you save a model with the same name, a new version of the model will be saved, so nothing will be overwritten. In this way, you can compare several versions of the same model - or create a model with a new name, if you prefer that.